In [1]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms
%matplotlib inline

2025-12-01 20:35:14.432441: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764621314.652907      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764621314.717669      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip -q data.zip

--2025-12-01 20:35:37--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-01T21%3A27%3A16Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-01T20%3A27%3A11Z&ske=2025-12-01T21%3A27%3A16Z&sks=b&skv=2018-11-09&sig=ox20rvTRjkbYK4KK1o8%2BYd9xwZG26BArNxizPZdPPS0%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDYyMzEzNywibmJmIjoxNzY0NjIxMzM3LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5i

The shape for input should be (3, 200, 200) (channels first format in PyTorch)

### Building a CNN

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class HairClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 1. Convolution: 3 → 32 channels, kernel (3x3)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        
        # 2. Max Pool (2x2)
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        # After conv + pool, compute flattened dimension:
        # Input: (3, 200, 200)
        # After conv: (32, 198, 198)
        # After pool: (32, 99, 99)
        self.flatten_dim = 32 * 99 * 99
        
        # 3. Linear layer with 64 neurons
        self.fc1 = nn.Linear(self.flatten_dim, 64)
        
        # 4. Final binary classifier output
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        
        x = x.view(-1, self.flatten_dim)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)   # output for binary classification
        return x

model = HairClassifier()
model

HairClassifier(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

### Q1. Which loss function you will use? 
Loss and optimization

In [5]:
criterion = nn.BCEWithLogitsLoss()

# optimizer = torch.optim.SGD(
#     model.parameters(),
#     lr=0.01,
#     momentum=0.9
# )

optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

Loss function used: nn.BCEWithLogitsLoss()

✅ nn.BCEWithLogitsLoss() (most correct)

⚠️ nn.MSELoss() (works but suboptimal)

❌ nn.CrossEntropyLoss() (only correct if you change architecture)

❌ nn.CosineEmbeddingLoss() (incorrect- It is for embedding similarity—not classification.)

### Q2. What's the total number of parameters of the model? You can use torchsummary or count manually.

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

HairClassifier(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [7]:
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
         MaxPool2d-2           [-1, 32, 99, 99]               0
            Linear-3                   [-1, 64]      20,072,512
            Linear-4                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 11.96
Params size (MB): 76.57
Estimated Total Size (MB): 89.00
----------------------------------------------------------------


Total params: 20,073,473
Trainable params: 20,073,473

#### Generators and Training

In [8]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

train_dataset = datasets.ImageFolder("data/train", transform=train_transforms)
test_dataset  = datasets.ImageFolder("data/test",  transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=20, shuffle=False)

print("Classes:", train_dataset.classes)

Classes: ['curly', 'straight']


In [9]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'test_acc': [], 'test_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    test_running_loss = 0.0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_epoch_loss = test_running_loss / len(test_dataset)
    test_epoch_acc = correct_test / total_test
    history['test_loss'].append(test_epoch_loss)
    history['test_acc'].append(test_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Test Loss: {test_epoch_loss:.4f}, Test Acc: {test_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6665, Acc: 0.6112, Test Loss: 0.6511, Test Acc: 0.6617
Epoch 2/10, Loss: 0.5702, Acc: 0.6787, Test Loss: 0.6332, Test Acc: 0.6318
Epoch 3/10, Loss: 0.5207, Acc: 0.7350, Test Loss: 0.6143, Test Acc: 0.6766
Epoch 4/10, Loss: 0.4773, Acc: 0.7600, Test Loss: 0.6049, Test Acc: 0.6617
Epoch 5/10, Loss: 0.4606, Acc: 0.7550, Test Loss: 0.7307, Test Acc: 0.5672
Epoch 6/10, Loss: 0.3954, Acc: 0.8275, Test Loss: 0.6412, Test Acc: 0.6866
Epoch 7/10, Loss: 0.2844, Acc: 0.8838, Test Loss: 0.8307, Test Acc: 0.6816
Epoch 8/10, Loss: 0.2885, Acc: 0.8788, Test Loss: 0.7052, Test Acc: 0.7114
Epoch 9/10, Loss: 0.1882, Acc: 0.9313, Test Loss: 0.9275, Test Acc: 0.6866
Epoch 10/10, Loss: 0.2585, Acc: 0.8912, Test Loss: 0.8158, Test Acc: 0.6915


### Q3. What is the median of training accuracy for all the epochs for this model?

In [10]:
np.median(history['acc'])

0.79375

### Question 4. Standard deviation of training loss for all the epochs 

In [11]:
np.std(history['loss'])

0.14617743215283388

### Data Augmentation

In [12]:
train_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

train_dataset = datasets.ImageFolder("data/train", transform=train_transforms)
test_dataset  = datasets.ImageFolder("data/test",  transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=20, shuffle=False)

print("Classes:", train_dataset.classes)

Classes: ['curly', 'straight']


In [13]:
num_epochs = 20
history = {'acc': [], 'loss': [], 'test_acc': [], 'test_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    test_running_loss = 0.0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_epoch_loss = test_running_loss / len(test_dataset)
    test_epoch_acc = correct_test / total_test
    history['test_loss'].append(test_epoch_loss)
    history['test_acc'].append(test_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Test Loss: {test_epoch_loss:.4f}, Test Acc: {test_epoch_acc:.4f}")

Epoch 1/20, Loss: 0.6307, Acc: 0.6600, Test Loss: 0.5963, Test Acc: 0.7114
Epoch 2/20, Loss: 0.5801, Acc: 0.6750, Test Loss: 0.5968, Test Acc: 0.7214
Epoch 3/20, Loss: 0.5374, Acc: 0.7175, Test Loss: 0.5870, Test Acc: 0.7015
Epoch 4/20, Loss: 0.5502, Acc: 0.7125, Test Loss: 0.5695, Test Acc: 0.7264
Epoch 5/20, Loss: 0.5250, Acc: 0.7412, Test Loss: 0.6758, Test Acc: 0.6816
Epoch 6/20, Loss: 0.5141, Acc: 0.7612, Test Loss: 0.5764, Test Acc: 0.7313
Epoch 7/20, Loss: 0.5217, Acc: 0.7362, Test Loss: 0.5756, Test Acc: 0.6866
Epoch 8/20, Loss: 0.4888, Acc: 0.7588, Test Loss: 0.5724, Test Acc: 0.7214
Epoch 9/20, Loss: 0.4989, Acc: 0.7475, Test Loss: 0.5452, Test Acc: 0.7313
Epoch 10/20, Loss: 0.4663, Acc: 0.7688, Test Loss: 0.5619, Test Acc: 0.7164
Epoch 11/20, Loss: 0.4634, Acc: 0.7800, Test Loss: 0.6009, Test Acc: 0.6617
Epoch 12/20, Loss: 0.4530, Acc: 0.7700, Test Loss: 0.5053, Test Acc: 0.7313
Epoch 13/20, Loss: 0.4420, Acc: 0.7812, Test Loss: 0.5256, Test Acc: 0.7463
Epoch 14/20, Loss: 0.

### Question 5. Mean of test loss for all the epochs 

In [14]:
np.mean(history['test_loss'])

0.5682816690399279

### Question 6. Average of test accuracy for the last 5 epochs

In [15]:
history['test_acc']

[0.7114427860696517,
 0.7213930348258707,
 0.7014925373134329,
 0.7263681592039801,
 0.681592039800995,
 0.7313432835820896,
 0.6865671641791045,
 0.7213930348258707,
 0.7313432835820896,
 0.7164179104477612,
 0.6616915422885572,
 0.7313432835820896,
 0.746268656716418,
 0.7313432835820896,
 0.7114427860696517,
 0.7711442786069652,
 0.7064676616915423,
 0.7014925373134329,
 0.6865671641791045,
 0.7860696517412935]

In [16]:
np.mean(history['test_acc'][-5:])

0.7303482587064677

In [17]:
average_test_acc = sum(history['test_acc'][-5:])/5
average_test_acc

0.7303482587064677